In [38]:
import array
import numpy as np
import soundfile as sf
from pydub import AudioSegment
import os

#df = pd.read_csv('cv-corpus-9.0-2022-04-27/ar/dev.tsv', sep='\t')

sound_path = 'cv-corpus-9.0-2022-04-27/ar/clips'
background_sound_path = 'background_sound/water-fountain-sound-effect.mp3'
background_sound = AudioSegment.from_mp3(background_sound_path) - 5
path = os.path.join('mixed_sound',background_sound_path.split('/')[-1].split('.')[0])


SNR_group = [0, 5, 10, 15]


for filename in os.listdir(sound_path):
    sound = AudioSegment.from_mp3(os.path.join(sound_path,filename))
    S_dBFS = sound.dBFS
    N_dBFS = background_sound.dBFS
    diff = np.abs(S_dBFS - N_dBFS)
    if S_dBFS > N_dBFS:
        background_sound += diff
    else:
        sound += diff
        
    for i in SNR_group:
        saved_path = os.path.join(path,'SNR_'+str(i)+'dB')
        if not os.path.exists(saved_path):
            os.makedirs(saved_path)

        sound += i
        #print(sound.dBFS, background_sound.dBFS)
        mixed_sound = sound.overlay(background_sound, position = 0) 
        mixed_sound = mixed_sound
        
        target_path = os.path.join(saved_path,filename)
            
        mixed_sound.export(target_path, format="mp3")



#print(mixed_sound.frame_rate)



<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pydub.audio_segment.AudioSegment'>
<class 'pyd

KeyboardInterrupt: 

In [15]:
from matplotlib import pyplot
import thinkdsp
import thinkplot
from pydub import AudioSegment
from pydub.utils import get_array_type

def trans_mp3_2_wav(filepath):
    sound = AudioSegment.from_mp3(filepath)
    sound.export(filepath.split('.')[0]+'.wav', format="wav")

def trans_wav_2_mp3(filepath):
    sound = AudioSegment.from_wav(filepath)
    sound.export(filepath.split('.')[0]+'.mp3', format="mp3")


trans_mp3_2_wav('guassian_noise.mp3')

'''
wave1=thinkdsp.read_wave("1.wav")
spectrum1=wave1.make_spectrum()
spectrum1.plot()
thinkplot.show()
'''

'\nwave1=thinkdsp.read_wave("1.wav")\nspectrum1=wave1.make_spectrum()\nspectrum1.plot()\nthinkplot.show()\n'

In [6]:
import librosa
import numpy as np
import soundfile as sf
from pydub import AudioSegment
import matplotlib.pyplot as plt


filename = "./common_voice_ar_19058307.mp3"

data, sr = librosa.load(filename, mono=True)
#data, samplerate = sf.read(filename, dtype='float32')

def add_guassian_noise(x, snr):
    P_signal = np.mean(x**2)    #power of signal
    k = np.sqrt(P_signal / 10 ** (snr / 10.0))  #amplitude
    
    noise = np.random.randn(len(x))
    return x + noise.astype('float32') * k

gussian_Augmentation = add_guassian_noise(x=data, snr=0)
sf.write('guassian_noise.wav', gussian_Augmentation,sr)
trans_wav_2_mp3('guassian_noise.wav')


/home/bao/miniconda3/envs/COD/lib/python3.6/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [3]:
import pyroomacoustics as pra
import numpy as np
import matplotlib.pyplot as plt
import librosa

rt_tgt = 0.5  # The time of the room reverberation
room_dim = [9, 7.5, 3.5]  # the size(9*7.5*3) of the room
filename = "common_voice_ar_19058307.mp3"
source = [2.5, 3.73, 1.76] # position of the source of the sound
mic_locs = np.c_[
    [6.3, 4.87, 1.2],  # mic 1
    [6.3, 4.93, 1.2],  # mic 2
] # position of the microphones

def reverberation(room_dim, rt_tgt, source, mic_locs, filename):
    
    e_absorption, max_order = pra.inverse_sabine(rt60_tgt, room_dim)   
    materials = pra.Material(energy_absorption="hard_surface")    # material of the wall
    max_order = 3 # the max number of reflection

    room = pra.ShoeBox(room_dim, fs=16000, materials=pra.Material(e_absorption), max_order=max_order)

    audio, _ = librosa.load(filename,sr=16000) 
    room.add_source(source, signal=audio, delay=0.3)

    room.add_microphone_array(mic_locs) 

    #Room Impulse Response
    room.compute_rir()

    room.simulate()

    room.mic_array.to_wav("./reverberation.wav", norm=True, bitdepth=np.float32,)

    
reverberation(room_dim, rt_tgt, source, mic_locs, filename)

ModuleNotFoundError: No module named 'pyroomacoustics'

In [2]:
#import soundpy as sp
import librosa
import soundfile as sf

audio, sr = librosa.load("common_voice_ar_19058307.mp3")

def dB_gain(wav, dB):
    wav_p = np.mean(wav ** 2)  # average power
    scalar = np.sqrt(10 ** (dB / 10) / (wav_p + np.finfo(np.float32).eps))
    wav *= scalar
    return wav, scalar

def augment_resample(wav, sr):
    resample_sr = np.random.uniform(sr)     # sampling from a uniform distribution
    print("target_sr", resample_sr)
    resample = librosa.resample(wav, orig_sr=sr, target_sr=resample_sr)
    resample = librosa.resample(resample, orig_sr=resample_sr, target_sr=sr)
    return resample

#resample = augment_resample(audio, sr)

def time_shift(x, shift):
    return np.roll(x, int(shift))

#shift_Augmentation = time_shift(audio, shift=len(audio)//10)
#sf.write('shift.wav', shift_Augmentation,sr)

def time_stretch(x, rate):
    # rate > 1 speed up
    # rate < 1 speed down
    return librosa.effects.time_stretch(x, rate)

#stretch_Augmentation = time_stretch(audio, rate=0.5)
#sf.write('stretch.wav', stretch_Augmentation,sr)

def pitch_shifting(x, sr, n_steps, bins_per_octave=12):
    return librosa.effects.pitch_shift(x, sr, n_steps, bins_per_octave=bins_per_octave)


up_3 = librosa.effects.pitch_shift(audio, sr=sr, n_steps=4, bins_per_octave=12)
sf.write('up_3.wav', up_3,sr)
down_3 = librosa.effects.pitch_shift(audio, sr=sr, n_steps=-6, bins_per_octave=12)
#down_3, _ = dB_gain(down_3, 5)
sf.write('down_3.wav', down_3,sr)
up_34 = librosa.effects.pitch_shift(audio, sr = sr, n_steps=3, bins_per_octave=24)



/home/bao/miniconda3/envs/audio/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
